In [ ]:
# import pathlib
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import re
#import random
#from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, roc_auc_score
from sklearn.preprocessing import QuantileTransformer
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe


import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV
#from string import ascii_letters
import seaborn as sns
import h5py as h5

#from string import ascii_letters

In [2]:
df =pd.read_pickle('../pickle_pd/second_pd_top_25.pkl')
features = list(df)[:-1]

In [3]:
qt = QuantileTransformer(random_state=0)
df[features] = qt.fit_transform(df[features])

In [ ]:
df.head()

In [5]:
 X_train, X_test, y_train, y_test = train_test_split( df[features], df['label'], test_size=0.4, random_state=42,shuffle = True)
 X_test, X_val, y_test, y_val = train_test_split( X_test, y_test, test_size=0.5, random_state=42,shuffle = True)

In [6]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        #'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 1,50,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.6,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': hp.choice('n_estimators', [300, 400, 500, 600]),
        'seed': 0
    }

In [7]:
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']),
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']), reg_lambda= space['reg_lambda'])
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, #eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(X_train)
    accuracy = accuracy_score(y_train, pred)
    pred_val = clf.predict(X_test)
    val_accuracy = accuracy_score(y_test, pred_val)
    delta_accuracy = accuracy - val_accuracy
    print ("SCORE:", accuracy)
    return {'loss': -val_accuracy, 'accuracy': accuracy, 'val_accuracy': val_accuracy, 'delta_accuracy':delta_accuracy, 'status': STATUS_OK }

In [8]:
best_hyperparams = {'colsample_bytree': 0.7835361011051029,
 'max_depth': 4.0,
 'min_child_weight': 7.0,
 'n_estimators': 3,
 'reg_alpha': 33.0,
 'reg_lambda': 0.46159501019482946}

In [9]:
trial_res = pd.read_csv('out.csv')

In [10]:
trial_res.head()

,colsample_bytree,max_depth,min_child_weight,n_estimators,reg_alpha,reg_lambda,train_accuracy,val_accuracy,delta_accuracy,Selection
0,0.691505,8.0,4.0,2,27.0,0.112930,0.811016,0.807617,0.003399,0.0
1,0.841174,7.0,0.0,2,46.0,0.597406,0.809089,0.807223,0.001866,0.0
2,0.758157,14.0,6.0,2,26.0,0.962773,0.811643,0.807559,0.004084,0.0
3,0.744317,14.0,1.0,0,1.0,0.783146,0.876875,0.796497,0.080378,0.0
4,0.710051,6.0,5.0,0,46.0,0.741247,0.807719,0.806147,0.001572,0.0


In [11]:
treail_res.t

NameError: name 'treail_res' is not defined

In [12]:
best_hyperparams

{'colsample_bytree': 0.7835361011051029,
 'max_depth': 4.0,
 'min_child_weight': 7.0,
 'n_estimators': 3,
 'reg_alpha': 33.0,
 'reg_lambda': 0.46159501019482946}

In [ ]:
sns.set(font_scale = 3)
sns.pairplot(trial_res, x_vars=['n_estimators', 'colsample_bytree', 'max_depth', 'min_child_weight', 'reg_alpha', 'reg_lambda' ], y_vars=['train_accuracy','val_accuracy','delta_accuracy'], kind='reg',  height=5, hue = 'Selection')
plt.show()

In [ ]:
sns.set(font_scale = 3)
sns.pairplot(trial_res, x_vars=['n_estimators', 'colsample_bytree', 'max_depth', 'min_child_weight', 'reg_alpha', 'reg_lambda' ], y_vars=['train_accuracy','val_accuracy','delta_accuracy'], kind='reg',  height=5, hue = 'Selection')

In [ ]:
best_hyperparams

In [ ]:
xgb_model=xgb.XGBClassifier(
                    n_estimators =[200,300,400, 500, 600][best_hyperparams['n_estimators']], max_depth = int(best_hyperparams['max_depth']),
                    reg_alpha = int(best_hyperparams['reg_alpha']),min_child_weight=int(best_hyperparams['min_child_weight']),
                    colsample_bytree=int(best_hyperparams['colsample_bytree']), reg_lambda= space['reg_lambda'])         
xgb_model.fit(X_train[features],y_train,
             eval_set = [( X_train, y_train), ( X_test, y_test)])
#y_pred = xgb_model.predict(X_val[features])

In [ ]:
xgb_model2=xgb.XGBClassifier(
                    n_estimators =200, max_depth = int(best_hyperparams['max_depth']),
                    reg_alpha = int(best_hyperparams['reg_alpha']),min_child_weight=int(best_hyperparams['min_child_weight']),
                    colsample_bytree=int(best_hyperparams['colsample_bytree']), reg_lambda= space['reg_lambda'])         
xgb_model2.fit(X_train[features],y_train,
             eval_set = [( X_train, y_train), ( X_test, y_test)])

In [ ]:
xgb_basic=xgb.XGBClassifier(n_estimators =180) 
evals_result = {}
xgb_basic.fit(X_train[features],y_train,
             eval_set = [( X_train, y_train), ( X_test, y_test)])

In [ ]:
pred_test = xgb_basic.predict(X_test)
accuracy_score(pred_test,y_test)

In [ ]:
history['validation_0']

In [ ]:
sns.set(font_scale = 1.5)
history  = xgb_model.evals_result()
history2  = xgb_basic.evals_result()
plt.plot(history['validation_0']['mlogloss'], label='train opt')
plt.plot(history['validation_1']['mlogloss'], label='val opt')
plt.plot(history2['validation_0']['mlogloss'], label='train basic')
plt.plot(history2['validation_1']['mlogloss'], label='val basic')
plt.ylabel('log loss')
plt.xlabel('epochs')
plt.title('XGB Training loss curve')
plt.legend()

In [ ]:
sns.set(font_scale = 1.5)
history  = xgb_model2.evals_result()
history2  = xgb_basic.evals_result()
plt.plot(history['validation_0']['mlogloss'], label='train opt')
plt.plot(history['validation_1']['mlogloss'], label='val opt')
plt.plot(history2['validation_0']['mlogloss'], label='train basic')
plt.plot(history2['validation_1']['mlogloss'], label='val basic')
plt.ylabel('log loss')
plt.xlabel('epochs')
plt.title('BDT Training loss curve')
plt.legend()

In [ ]:
history.keys()

In [ ]:
history['validation_0']

In [ ]:
plt.plot(history['validation_0']['mlogloss'], label='train')
plt.plot(history['validation_1']['mlogloss'], label='test')
plt.legend()

In [ ]:
y_pred =  xgb_model.predict(X_val)
y_pred_train = xgb_model.predict(X_train)